In [1]:
import os

import matplotlib as mpl
import numpy as np

# from cellpose import plot


%matplotlib inline
mpl.rcParams["figure.dpi"] = 300


np.random.seed(0)


import numpy as np
from skimage import data

In [2]:
import scipy

scipy.__version__

'1.7.3'

In [2]:
# input_dir="/cluster/work/treutlein/DATA/imaging/viventis/20210518_181629_Act-GFP_TubRFP_Cysts/"

In [3]:
import json
import os

import ipywidgets as widgets
from jupyter_bbox_widget import BBoxWidget

In [5]:
# annotation_run=70
annotation_run = 1
run_name = f"Run_{annotation_run}"
slice_dir = f"annotation_slices/AGAR/{run_name}/"
# slice_dir=f'annotation_slices/MultiMosaic/{run_name}/'

In [6]:
annotations_path = f"{slice_dir}annotations.json"

In [7]:
# path = f'annotation_slices/{run_name}/'
files = sorted(os.listdir(slice_dir))

annotations = {}

In [8]:
# a progress bar to show how far we got
w_progress = widgets.IntProgress(value=0, max=len(files), description="Progress")
# the bbox widget
w_bbox = BBoxWidget(
    image=os.path.join(slice_dir, files[0]), classes=["organoid", "background", "lumen"]
)


from pathlib import Path

my_file = Path(annotations_path)
if my_file.is_file():
    f = open(annotations_path)
    data = json.load(f)
    json_availible = True
else:
    json_availible = False
# combine widgets into a container
w_container = widgets.VBox(
    [
        w_progress,
        w_bbox,
    ]
)

# when Skip button is pressed we move on to the next file


def on_skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = os.path.join(slice_dir, image_file)
    # here we assign an empty list to bboxes but
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    if json_availible:
        try:
            w_bbox.bboxes = data[image_file]
        except:
            w_bbox.bboxes = []
    else:
        w_bbox.bboxes = []


w_bbox.on_skip(on_skip)

# when Submit button is pressed we save current annotations
# and then move on to the next file


def on_submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open(annotations_path, "w") as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    on_skip()


w_bbox.on_submit(on_submit)
w_container